In [1]:
# bt_ema_pullback_v4.ipynb

import sys
sys.path.append("D:/trading")  # o el path donde está tu carpeta my_modules
# ✅ CELDA 1: Preparación del entorno
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from my_modules.estrategias.v4 import ema_pullback_v4

# Cargar históricos locales
ruta_historicos = "D:/trading/data/historic"
historicos = {}
for archivo in os.listdir(ruta_historicos):
    if archivo.endswith(".parquet"):
        ticker = archivo.replace(".parquet", "")
        df = pd.read_parquet(os.path.join(ruta_historicos, archivo))
        if len(df) >= 25:
            historicos[ticker] = df

print(f"Históricos cargados: {len(historicos)} símbolos")




Históricos cargados: 48 símbolos


In [2]:
# ✅ CELDA 2: Loop de generación y simulación de señales
resultados = []

for ticker, df in tqdm(historicos.items()):
    df_signals = ema_pullback_v4.generar_senales(
        df,
        usar_filtro_volatilidad=True,
        atr_threshold=0.010,
        rebote_required=True,
        tendencia_required=True,
        debug=False
    )

    df_signals = df_signals[df_signals["signal"] != "hold"]
    if df_signals.empty:
        continue

    df_precio = df.copy()
    df_precio["fecha"] = pd.to_datetime(df_precio["fecha"])
    df_signals["fecha"] = pd.to_datetime(df_signals["fecha"])
    df_merged = df_precio.merge(df_signals, on="fecha")

    for _, row in df_merged.iterrows():
        fecha_entrada = row["fecha"]
        precio_entrada = row["close"]
        df_rango = df_precio[(df_precio["fecha"] > fecha_entrada) & (df_precio["fecha"] <= fecha_entrada + pd.Timedelta(days=7))]
        if df_rango.empty:
            continue

        tipo_salida = "TIMEOUT"
        fila_salida = df_rango.iloc[-1]

        for _, f in df_rango.iterrows():
            if row["signal"] == "buy":
                if f["high"] >= precio_entrada * 1.05:
                    tipo_salida = "TP"; fila_salida = f; break
                if f["low"] <= precio_entrada * 0.97:
                    tipo_salida = "SL"; fila_salida = f; break
            elif row["signal"] == "sell":
                if f["low"] <= precio_entrada * 0.95:
                    tipo_salida = "TP"; fila_salida = f; break
                if f["high"] >= precio_entrada * 1.03:
                    tipo_salida = "SL"; fila_salida = f; break

        precio_salida = fila_salida["close"]
        dias = (fila_salida["fecha"] - fecha_entrada).days
        resultado = precio_salida - precio_entrada if row["signal"] == "buy" else precio_entrada - precio_salida
        ret_pct = (precio_salida / precio_entrada - 1) * (1 if row["signal"] == "buy" else -1)
        log_ret = np.log(precio_salida / precio_entrada) * (1 if row["signal"] == "buy" else -1)

        resultados.append({
            "ticker": ticker,
            "fecha_entrada": fecha_entrada,
            "fecha_salida": fila_salida["fecha"],
            "dias": dias,
            "precio_entrada": precio_entrada,
            "precio_salida": precio_salida,
            "resultado": resultado,
            "resultado_pct": ret_pct,
            "log_ret": log_ret,
            "f_win": int(resultado > 0),
            "tipo_salida": tipo_salida,
            "estrategia": "ema_pullback_v4",
            "signal": row["signal"]
        })



  0%|                                                                                           | 0/48 [00:00<?, ?it/s]D:\trading\my_modules\estrategias\v4\ema_pullback_v4.py:90: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["prev_buy"] = pd.Series(df["buy_cond"].shift(1)).fillna(False).astype(bool)
  2%|█▋                                                                                 | 1/48 [00:00<00:27,  1.73it/s]D:\trading\my_modules\estrategias\v4\ema_pullback_v4.py:90: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["prev_buy"] = pd.Series(df["

In [3]:
# ✅ CELDA 3: Consolidar y evaluar

import seaborn as sns
import matplotlib.pyplot as plt

df_operaciones = pd.DataFrame(resultados)
print(f"Operaciones simuladas: {len(df_operaciones):,}")

resumen = df_operaciones.groupby("estrategia").agg({
    "resultado": ["sum", "mean", "count"],
    "f_win": "mean",
    "dias": "mean",
    "log_ret": "mean"
})
resumen.columns = ["total_profit", "avg_profit", "n_trades", "winrate", "avg_duracion", "avg_logret"]
resumen["score"] = resumen["avg_profit"] * resumen["winrate"]
resumen = resumen.sort_values("score", ascending=False).round(4)
resumen


Operaciones simuladas: 19,346


,total_profit,avg_profit,n_trades,winrate,avg_duracion,avg_logret,score
estrategia,,,,,,,
ema_pullback_v4,808.4555,0.0418,19346,0.4824,5.68,0.0005,0.0202
